In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import filters, color, graph, img_as_float
from skimage.segmentation import watershed, active_contour, felzenszwalb
from scipy import ndimage as ndi
from sklearn.cluster import MeanShift, estimate_bandwidth

# ------------------- Load Uploaded Image -------------------
image_path = "/content/jackie.webp"  # Replace with your uploaded image
img = Image.open(image_path).convert('L')
image = img_as_float(np.array(img))  # Convert to float [0,1]

# Resize for speed if large
if image.shape[0] > 400:
    image = image[:400, :400]

# ==================== a. Watershed Segmentation (Region-Based) ====================
gradient = filters.sobel(image)
markers = np.zeros_like(image, dtype=np.int32)  # Must be integer
markers[image < 0.3] = 1
markers[image > 0.7] = 2
labels_ws = watershed(gradient, markers)

# ==================== b. Mean Shift Segmentation (Cluster-Based) ====================
X, Y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
features = np.column_stack((X.ravel(), Y.ravel(), image.ravel()))
bandwidth = estimate_bandwidth(features, quantile=0.1, n_samples=500)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(features)
labels_ms = ms.labels_.reshape(image.shape)

# ==================== c. Active Contour (Model-Based) ====================
s = np.linspace(0, 2*np.pi, 400)
x = image.shape[1]/2 + (image.shape[1]/3)*np.cos(s)
y = image.shape[0]/2 + (image.shape[0]/3)*np.sin(s)
init = np.array([x, y]).T

snake = active_contour(filters.gaussian(image, 3),
                       init, alpha=0.015, beta=10, gamma=0.001)

# ==================== d. Normalized Cut Segmentation (Graph-Based) ====================
labels_fz = felzenszwalb(image, scale=100)
image_rgb = color.gray2rgb(image)  # Required for RAG
g = graph.rag_mean_color(image_rgb, labels_fz)
labels_nc = graph.cut_normalized(labels_fz, g)

# ==================== Display Results ====================
plt.figure(figsize=(12, 8))

plt.subplot(2,2,1)
plt.title("a. Watershed Segmentation")
plt.imshow(color.label2rgb(labels_ws, image=image))
plt.axis("off")

plt.subplot(2,2,2)
plt.title("b. Mean Shift Segmentation")
plt.imshow(color.label2rgb(labels_ms, image=image))
plt.axis("off")

plt.subplot(2,2,3)
plt.title("c. Active Contour Segmentation")
plt.imshow(image, cmap='gray')
plt.plot(init[:,0], init[:,1], '--r', lw=2)
plt.plot(snake[:,0], snake[:,1], '-b', lw=2)
plt.axis("off")

plt.subplot(2,2,4)
plt.title("d. Normalized Cut Segmentation")
plt.imshow(color.label2rgb(labels_nc, image=image_rgb))
plt.axis("off")

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/jackie.webp'